In [3]:
import boto3
import datetime
import re
import numpy as np

In [28]:
logStreamName = get_response_list(1)

In [8]:
cloudwatch = boto3.client('logs',region_name='us-east-1')
response = cloudwatch.get_log_events(
        logGroupName='/aws/lambda/blackjack_prototype',
        logStreamName=logStreamName[0],
        startFromHead=True)

In [20]:
trigger_words = ["You bet","you bet","You hit","you hit","You stand","you stand"]

In [41]:
def get_timestamps_labels(logStreamName,
                   trigger_words = ["You bet","you bet","You hit","you hit","You stand","you stand"],
                   logGroupName='/aws/lambda/blackjack_prototype',
                   lag=0):
    cloudwatch = boto3.client('logs',region_name='us-east-1')

    response = cloudwatch.get_log_events(
        logGroupName='/aws/lambda/blackjack_prototype',
        logStreamName=logStreamName,
        startFromHead=True
    )

    M_flag = 0
    trigger_words = ["You bet","you bet","You hit","you hit","You stand","you stand"]
    back = 0
    ret_dict={}
    lag=0
    start_time = datetime.datetime.fromtimestamp(round(response['events'][0]['timestamp']/1000))
    for each in response['events']:
        try:

            if "Mistake made" in each['message']:
                M_flag = 1


            Alexa_s = re.findall(r'<speak>([\s\S]*?)<\/speak>',each['message'])[0]
            t_converted = datetime.datetime.fromtimestamp(round(each['timestamp'] / 1000))
            timestamp_1 = t_converted-start_time+datetime.timedelta(seconds=lag)

            if back == 1:
                ret_dict[key] = str(timestamp_1)
                back = 0

            for t_word in trigger_words:
                if t_word in Alexa_s:
                    if M_flag == 1:
                        key = str("Mistake made"+"-"+str(timestamp_1))
                        M_flag = 0
                    else:
                        key = str("No Mistake made"+"-"+str(timestamp_1))
                    
                    key = t_word+"-"+ key
                    ret_dict[key]=None    
                    back = 1

        except IndexError:
            continue
    return ret_dict

In [6]:
def get_response_list(N_interactions):
    cloudwatch = boto3.client('logs',region_name='us-east-1')
    response = cloudwatch.describe_log_streams(
        logGroupName='/aws/lambda/blackjack_prototype',
        orderBy='LastEventTime',
        descending=True,
        limit=N_interactions
    )
    response_list = [i['logStreamName'] for i in response['logStreams']]
    
    return response_list

In [36]:
def get_timestamps(logStreamName,
                   trigger_words = ["You bet","you bet","You hit","you hit","You stand","you stand"],
                   logGroupName='/aws/lambda/blackjack_prototype',
                   lag=0):
    cloudwatch = boto3.client('logs',region_name='us-east-1')

    response = cloudwatch.get_log_events(
        logGroupName='/aws/lambda/blackjack_prototype',
        logStreamName=logStreamName,
        startFromHead=True
    )


    trigger_words = trigger_words
    back = 0
    lag = lag
    ret_dict = {}
    start_time = datetime.datetime.fromtimestamp(round(response['events'][0]['timestamp']/1000))
    for each in response['events']:
        try:
            Alexa_s = re.findall(r'<speak>([\s\S]*?)<\/speak>',each['message'])[0]
            t_converted = datetime.datetime.fromtimestamp(round(each['timestamp'] / 1000))
            timestamp_1 = t_converted-start_time+datetime.timedelta(seconds=lag)
            if back == 1:
                ret_dict[key] = str(timestamp_1)
                back = 0
            for t_word in trigger_words:
                if t_word in Alexa_s:
                    ret_dict[str(timestamp_1)]=None
                    key = str(timestamp_1)
                    back = 1
        except IndexError:
            continue

    return ret_dict

In [37]:
get_timestamps(logStreamName='2019/03/05/[$LATEST]df74413a62cb47cf923cf8a8788a0d23',lag = 8)

{'0:01:00': '0:01:17',
 '0:01:17': '0:01:27',
 '0:01:27': '0:01:42',
 '0:01:47': '0:02:04',
 '0:02:04': '0:02:35',
 '0:02:40': '0:03:08',
 '0:03:14': '0:03:36',
 '0:03:36': '0:03:46',
 '0:03:46': '0:03:56',
 '0:03:56': '0:04:13',
 '0:04:19': '0:04:40',
 '0:04:40': '0:05:03',
 '0:05:13': '0:05:34',
 '0:05:34': '0:06:05',
 '0:06:10': '0:06:28',
 '0:06:28': '0:07:05',
 '0:07:10': '0:07:27',
 '0:07:27': '0:07:59',
 '0:08:05': '0:08:23',
 '0:08:23': '0:09:06',
 '0:09:13': '0:09:30',
 '0:09:30': '0:09:40',
 '0:09:40': '0:09:51',
 '0:09:51': '0:10:06',
 '0:10:11': '0:10:29',
 '0:10:29': None}

In [169]:
for i in get_response_list(1):
    print(get_timestamps(logStreamName=i,lag=8))

{'0:01:00': '0:01:17', '0:01:17': '0:01:27', '0:01:27': '0:01:42', '0:01:47': '0:02:04', '0:02:04': '0:02:35', '0:02:40': '0:03:08', '0:03:14': '0:03:36', '0:03:36': '0:03:46', '0:03:46': '0:03:56', '0:03:56': '0:04:13', '0:04:19': '0:04:40', '0:04:40': '0:05:03', '0:05:13': '0:05:34', '0:05:34': '0:06:05', '0:06:10': '0:06:28', '0:06:28': '0:07:05', '0:07:10': '0:07:27', '0:07:27': '0:07:59', '0:08:05': '0:08:23', '0:08:23': '0:09:06', '0:09:13': '0:09:30', '0:09:30': '0:09:40', '0:09:40': '0:09:51', '0:09:51': '0:10:06', '0:10:11': '0:10:29', '0:10:29': None}


'2019_03_05_788a0d23'

In [186]:
i[-8:]

'788a0d23'

In [185]:
i

'2019/03/05/[$LATEST]df74413a62cb47cf923cf8a8788a0d23'